# Quantum circuit of Shor's Code using Cirq
Cirq does not have barrier command like Qiskit.

Instead it always packs whatever you append into the “tightest” sequence.

In [ ]:
!pip install cirq

In [ ]:
import cirq
qubits = cirq.LineQubit.range(9)

simulator = cirq.Simulator()

# Design the quantum circuit

By default, all qubits are $\lvert 0 \rangle$, we use Hadamard gate to change $q_{0}$ from $\lvert 0 \rangle$ to $\lvert + \rangle$.

In [ ]:
circuit = cirq.Circuit()
circuit.append(cirq.H(qubits[0]))
circuit

0: ───H───

Use CNOT gates to entangle $q_{0}$ with $q_{3}$ and $q_{6}$ to protect bit flip error.

In [ ]:
circuit.append([
    cirq.CNOT(qubits[0], qubits[3]),
    cirq.CNOT(qubits[0], qubits[6])
])
circuit

0: ───H───@───@───
          │   │
3: ───────X───┼───
              │
6: ───────────X───

Map to the Hadamard basis and entangle two qubits to each qubits to protect phase flip error.

In [ ]:
circuit.append([cirq.H(qubits[i]) for i in [0, 3, 6]])
circuit += cirq.Moment()
circuit.append([
    cirq.CNOT(qubits[0], qubits[1]), cirq.CNOT(qubits[0], qubits[2])
])
circuit.append([
    cirq.CNOT(qubits[3], qubits[4]), cirq.CNOT(qubits[3], qubits[5])
])
circuit.append([
    cirq.CNOT(qubits[6], qubits[7]), cirq.CNOT(qubits[6], qubits[8])
])
circuit += cirq.Moment()
circuit

┌──┐
0: ───H───@────@─────H───@───@───────
          │    │         │   │
1: ───────┼────┼─────────X───┼───────
          │    │             │
2: ───────┼────┼─────────────X───────
          │    │
3: ───────X────┼H────@───@───────────
               │     │   │
4: ────────────┼─────X───┼───────────
               │         │
5: ────────────┼─────────X───────────
               │
6: ────────────X─────H───@───@───────
                         │   │
7: ──────────────────────X───┼───────
                             │
8: ──────────────────────────X───────
              └──┘

Introduce an arbitrary error, which could happen on any qubit, let's take XZ on $q_{0}$ for example.

In [ ]:
circuit.append([cirq.X(qubits[0]), cirq.Z(qubits[0])])
circuit

┌──┐
0: ───H───@────@─────H───@───@───X───Z───
          │    │         │   │
1: ───────┼────┼─────────X───┼───────────
          │    │             │
2: ───────┼────┼─────────────X───────────
          │    │
3: ───────X────┼H────@───@───────────────
               │     │   │
4: ────────────┼─────X───┼───────────────
               │         │
5: ────────────┼─────────X───────────────
               │
6: ────────────X─────H───@───@───────────
                         │   │
7: ──────────────────────X───┼───────────
                             │
8: ──────────────────────────X───────────
              └──┘

Apply majority-vote correction for phase flip and map back to original computational basis.

In [ ]:
circuit.append([
    cirq.CNOT(qubits[0], qubits[1]), cirq.CNOT(qubits[0], qubits[2]), cirq.CCX(qubits[1], qubits[2], qubits[0])
])
circuit.append([
    cirq.CNOT(qubits[3], qubits[4]), cirq.CNOT(qubits[3], qubits[5]), cirq.CCX(qubits[4], qubits[5], qubits[3])
])
circuit.append([
    cirq.CNOT(qubits[6], qubits[7]), cirq.CNOT(qubits[6], qubits[8]), cirq.CCX(qubits[7], qubits[8], qubits[6])
])
circuit.append([cirq.H(qubits[i]) for i in [0, 3, 6]])
circuit

┌──┐
0: ───H───@────@─────H───@───@───X───Z───@───@───X───H───
          │    │         │   │           │   │   │
1: ───────┼────┼─────────X───┼───────────X───┼───@───────
          │    │             │               │   │
2: ───────┼────┼─────────────X───────────────X───@───────
          │    │
3: ───────X────┼H────@───@───@───@───X───H───────────────
               │     │   │   │   │   │
4: ────────────┼─────X───┼───X───┼───@───────────────────
               │         │       │   │
5: ────────────┼─────────X───────X───@───────────────────
               │
6: ────────────X─────H───@───@───@───@───X───H───────────
                         │   │   │   │   │
7: ──────────────────────X───┼───X───┼───@───────────────
                             │       │   │
8: ──────────────────────────X───────X───@───────────────
              └──┘

Apply majority-vote correction for bit flip.

In [ ]:
circuit.append([
    cirq.CNOT(qubits[0], qubits[3]), cirq.CNOT(qubits[0], qubits[6]), cirq.CCX(qubits[3], qubits[6], qubits[0])
])
circuit

┌──┐
0: ───H───@────@─────H───@───@───X───Z───@───@───X───H───@───@───X───
          │    │         │   │           │   │   │       │   │   │
1: ───────┼────┼─────────X───┼───────────X───┼───@───────┼───┼───┼───
          │    │             │               │   │       │   │   │
2: ───────┼────┼─────────────X───────────────X───@───────┼───┼───┼───
          │    │                                         │   │   │
3: ───────X────┼H────@───@───@───@───X───H───────────────X───┼───@───
               │     │   │   │   │   │                       │   │
4: ────────────┼─────X───┼───X───┼───@───────────────────────┼───┼───
               │         │       │   │                       │   │
5: ────────────┼─────────X───────X───@───────────────────────┼───┼───
               │                                             │   │
6: ────────────X─────H───@───@───@───@───X───H───────────────X───@───
                         │   │   │   │   │
7: ──────────────────────X───┼───X───┼───@───────────────────────────
                             │       │   │
8: ──────────────────────────X───────X───@───────────────────────────
              └──┘

Measure the qubit $q_{0}$ and write into the classical bit $c$.

In [ ]:
circuit.append(cirq.measure(qubits[0], key='m'))
circuit

┌──┐
0: ───H───@────@─────H───@───@───X───Z───@───@───X───H───@───@───X───M('m')───
          │    │         │   │           │   │   │       │   │   │
1: ───────┼────┼─────────X───┼───────────X───┼───@───────┼───┼───┼────────────
          │    │             │               │   │       │   │   │
2: ───────┼────┼─────────────X───────────────X───@───────┼───┼───┼────────────
          │    │                                         │   │   │
3: ───────X────┼H────@───@───@───@───X───H───────────────X───┼───@────────────
               │     │   │   │   │   │                       │   │
4: ────────────┼─────X───┼───X───┼───@───────────────────────┼───┼────────────
               │         │       │   │                       │   │
5: ────────────┼─────────X───────X───@───────────────────────┼───┼────────────
               │                                             │   │
6: ────────────X─────H───@───@───@───@───X───H───────────────X───@────────────
                         │   │   │   │   │
7: ──────────────────────X───┼───X───┼───@────────────────────────────────────
                             │       │   │
8: ──────────────────────────X───────X───@────────────────────────────────────
              └──┘

# Simulate the quantum circuit
The result shows the distribution of 10000 total counts, which meets expectation.

In [ ]:
result = simulator.run(circuit, repetitions=10000)
counts = result.histogram(key='m')
print('Result:', counts)

Result: Counter({1: 5036, 0: 4964})
